In [214]:
import pandas as pd
from highcharts import Highchart
from datetime import datetime as dt

from IPython.core.display import display, HTML

In [216]:
H = Highchart()

timeline_data = pd.read_csv('timeline.txt')

options = {
    'chart': {
        'type': 'columnrange',
        'inverted': True,
        'zoomType': 'y'
    },
    'title': {
        'text': 'BepiColombo MPO Timeline'
    },
    'xAxis': {
        'categories': list(set(timeline_data['Instrument'].values.tolist()))
    },
    'yAxis': {
        'type': 'datetime'
    },
    'tooltip': {
         'formatter': "function () {return Highcharts.dateFormat('%e %B %H:%M', this.point.low) + \
         ' - ' + Highcharts.dateFormat('%e %B %H:%M', this.point.high) + \
         '<br><b>Mode: </b>' + this.point.mode + \
         '<br><b>Power: </b>' + this.point.power + \
         '<br><b>Data Rate: </b>' + this.point.data_rate;}"
    },
    'plotOptions': {
        'columnrange': {
            'grouping': False
        }
    }
}

H.set_dict_options(options)

grouped = timeline_data.groupby('Instrument')
grouped = [grouped.get_group(x) for x in grouped.groups]

for level, frame in enumerate(grouped):
    df = {}
    df['name'] = frame['Instrument'].values[0]
    df['data'] = []
    
    for row in frame.itertuples():
        block = {}
        block['x'] = level
        st = dt.strptime(row[2], '%Y-%m-%d %H:%M')
        st = int((st-dt(1970,1,1)).total_seconds()*1000)
        en = dt.strptime(row[3], '%Y-%m-%d %H:%M')
        en = int((en-dt(1970,1,1)).total_seconds()*1000)
        block['low'] = st
        block['high'] = en
        block['mode'] = row[4]
        block['power'] = row[5]
        block['data_rate'] = row[6]
        df['data'].append(block)
    
    H.add_data_set(df['data'], 'columnrange', df['name'] )
    
H